In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings

In [2]:
#Downloading price data.
import requests
import io
from datetime import datetime

interval = '1m'
symbol = 'BTCUSDT'

if interval not in ['1m', '3m', '5m', '15m', '30m', '1h', '2h', '4h', '6h', '8h', '12h', '1d', '3d', '1w', '1M']:
    raise ValueError('Invalid time interval.')
     
request = requests.get('https://api.binance.com/api/v1/klines?symbol=' + symbol + '&interval=' + interval)

data = pd.read_json(io.StringIO(request.text))
data.columns =  ['Open Time', 'Open', 'High', 'Low', 'Close', 'Volume', 'Close Time', 'Quote Asset Volume',
                 'Number of Trades', 'Taker Buy Base Asset Volume', 'Taker Buy Quote Asset Volume', 'Ignore']
data = data[['Open Time', 'Open']]
data['Open Time'] = data['Open Time'] / 1000
data['Open Time'] = data['Open Time'].apply(datetime.fromtimestamp)

series = pd.Series(data['Open'].values, index = data['Open Time'])

In [3]:
#ARIMA 
from statsmodels.tsa.arima_model import ARIMA
from sklearn import metrics 
warnings.filterwarnings("ignore", message =
                        "No frequency information was provided, so inferred frequency T will be used")

model_arima = ARIMA(series, (5, 1, 1))
model_arima_fit = model_arima.fit(disp = 0)
forecast = model_arima_fit.forecast()
pred_arima, ci_95 = str(forecast[0][0]), str(1.96*forecast[1][0])

print("The 95% confidence interval for the next price is " + pred_arima + " ± " + ci_95)

The 95% confidence interval for the next price is 6617.135529171889 ± 4.444893248399312


In [4]:
#RNN
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from sklearn.preprocessing import MinMaxScaler

scaler = MinMaxScaler(feature_range = (-1, 1))
series_scaled = scaler.fit_transform(series.values.reshape(500, 1))

df = pd.DataFrame(series_scaled)
df[1] = df[0].shift(-1)
df.dropna(inplace = True)

X, y = df[0], df[1]
X, y = X.values.reshape(499, 1, 1), y.values.reshape(499, 1)
X_test = series_scaled[-1:].reshape(1,1,1)

model_rnn = Sequential()
model_rnn.add(LSTM(256, return_sequences = True, input_shape = (1, 1)))
model_rnn.add(Dropout(0.5))
model_rnn.add(LSTM(256))
model_rnn.add(Dropout(0.5))
model_rnn.add(Dense(1))
model_rnn.compile(loss = 'mse', optimizer = 'adam')

model_rnn.fit(X, y, batch_size = 6, epochs = 12, shuffle = False, validation_split = 0.1, verbose = False)
pred_rnn_scaled = model_rnn.predict(X_test)
pred_unscaled = scaler.inverse_transform(pred_rnn_scaled)
pred_rnn = str(pred_unscaled[0][0])

print("The neural network predicts the next price to be " + pred_rnn)

The neural network predicts the next price to be 6616.754
